In [1]:
!pip install transformers


     |████████████████████████████████| 3.8 MB 4.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
     |████████████████████████████████| 6.5 MB 35.3 MB/s 
     |████████████████████████████████| 895 kB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import sys

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 2022WI folder and put all the files under A4 folder, then "2022WI/A4"
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "487/propaganda_detection"
GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))


# Add to sys so we can import .py files.
sys.path.append(GOOGLE_DRIVE_PATH)

['readme.md', 'data_logical_fallacy', 'data_proppy', 'ingest_data_test.ipynb', 'naieve_bayes_baseline.ipynb', 'dataset.py', '.git', 'model.py', 'Copy of ingest_data_test.ipynb', '__pycache__', '.idea']


In [4]:
from dataset import FallacyDataset
from pathlib import Path

# train_dataset = ProppyDataset(Path(GOOGLE_DRIVE_PATH) / "data_proppy" / "proppy_1.0.train.tsv")
# test_dataset = ProppyDataset(Path(GOOGLE_DRIVE_PATH) / "data_proppy" / "proppy_1.0.test.tsv")

train_dataset = FallacyDataset(Path(GOOGLE_DRIVE_PATH) / "data_logical_fallacy" / "edu_train.csv")
test_dataset = FallacyDataset(Path(GOOGLE_DRIVE_PATH) / "data_logical_fallacy" / "edu_test.csv")

In [88]:
train_dataset[0]

drive/My Drive/487/propaganda_detection/dataset.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}


{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  2194,  1005,  1055, 14558,  1000,  5987,  2062,  1012,  3477,
          2625,  1012,  1000,   102,     0,     0,     0,     0,     0,     0,
     

In [5]:
import numpy as np
np.unique([x['label'] for x in train_dataset], return_counts=True)

drive/My Drive/487/propaganda_detection/dataset.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([130, 174, 158, 134, 114, 319, 225, 106,  39, 121, 106, 112, 110]))

In [8]:
from model import model
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    # print(pred.label_ids)
    # print(pred.predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    learning_rate=1e-4,
    weight_decay=0,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [9]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1848
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2310
drive/My Drive/487/propaganda_detection/dataset.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,2.500315,0.176667,0.122529,0.138152,0.158795
2,No log,2.952849,0.203333,0.152334,0.150170,0.161479
3,1.909800,4.004936,0.183333,0.149625,0.154409,0.160655
4,1.909800,5.155292,0.183333,0.153141,0.152476,0.189930
5,0.530100,6.038149,0.183333,0.142581,0.144152,0.160623
6,0.530100,6.807018,0.170000,0.136886,0.137130,0.158430
7,0.112900,6.920737,0.183333,0.144378,0.145354,0.161077
8,0.112900,6.960622,0.193333,0.158055,0.161635,0.170068
9,0.022100,7.079723,0.183333,0.151939,0.154322,0.166285


***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


[ 0  1  2  3  1  0  0  1  4  5  3  1  6  7  1  8  7  2  6  8  5  7  9  5
  0  9  6  6  1  6  1  5  5  4 10  9  3  7  1  7  4  8  6  8  7 10  8  6
  4  2  5  1 10 11  9  5  8  1  1 12  2  1  6 12  8  6  7  9  8  4 12  4
  7  9  6  8  7  2 12  8  2  2 10  6 10 12  1  6  0  6 12  3  4  4  6  3
  7 10  9  3  6 10  6  2  1  3  0  0  3  5  8  2  1  1  1 12 10  9  3  6
  2  5  1  1  8  9  6 11  1  4  9  1  7  4  1  2  1  6  1  6  1  0  6  1
  1  9  1  7  9  0  4  6  1  3  6  7  4  6  7  9  9  6 10  1  1  6  0  4
  0  1 10 10  2  3  1  7  1  7  9  3  9  0  5  9 12  7 10  6  1  1  5  9
  0 10 11  7  1  1  1  1  6  1 10  7  9  7  3  6  7  1  7  4  7  0  4  6
  9  6  7 12  1  3  6  1  8 10 11  6  1 10  8 10  1  2  5 10 10  9  5  6
 10  1  9  9  3  5  1  0  1  6  5 10  9 12  8  1  4 12  0  3  1  0  5  7
  0  5  2  1  0 10  1 11 12  3 10  6  1  4  7  1  0  1  3  1  6  6  6  1
  1  6  7  1  7  6  5  7  0  7  1  5]
[[ 0.4111157  -0.05412817  0.36585614 ... -0.4129431  -0.4097272
  -0.50862986]
 [-0.0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
drive/My Drive/487/propaganda_detection/dataset.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,2.500315,0.176667,0.122529,0.138152,0.158795
2,No log,2.952849,0.203333,0.152334,0.150170,0.161479
3,1.909800,4.004936,0.183333,0.149625,0.154409,0.160655
4,1.909800,5.155292,0.183333,0.153141,0.152476,0.189930
5,0.530100,6.038149,0.183333,0.142581,0.144152,0.160623
6,0.530100,6.807018,0.170000,0.136886,0.137130,0.158430
7,0.112900,6.920737,0.183333,0.144378,0.145354,0.161077
8,0.112900,6.960622,0.193333,0.158055,0.161635,0.170068
9,0.022100,7.079723,0.183333,0.151939,0.154322,0.166285
10,0.022100,7.170796,0.176667,0.145138,0.144581,0.161819


***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


[ 0  1  2  3  1  0  0  1  4  5  3  1  6  7  1  8  7  2  6  8  5  7  9  5
  0  9  6  6  1  6  1  5  5  4 10  9  3  7  1  7  4  8  6  8  7 10  8  6
  4  2  5  1 10 11  9  5  8  1  1 12  2  1  6 12  8  6  7  9  8  4 12  4
  7  9  6  8  7  2 12  8  2  2 10  6 10 12  1  6  0  6 12  3  4  4  6  3
  7 10  9  3  6 10  6  2  1  3  0  0  3  5  8  2  1  1  1 12 10  9  3  6
  2  5  1  1  8  9  6 11  1  4  9  1  7  4  1  2  1  6  1  6  1  0  6  1
  1  9  1  7  9  0  4  6  1  3  6  7  4  6  7  9  9  6 10  1  1  6  0  4
  0  1 10 10  2  3  1  7  1  7  9  3  9  0  5  9 12  7 10  6  1  1  5  9
  0 10 11  7  1  1  1  1  6  1 10  7  9  7  3  6  7  1  7  4  7  0  4  6
  9  6  7 12  1  3  6  1  8 10 11  6  1 10  8 10  1  2  5 10 10  9  5  6
 10  1  9  9  3  5  1  0  1  6  5 10  9 12  8  1  4 12  0  3  1  0  5  7
  0  5  2  1  0 10  1 11 12  3 10  6  1  4  7  1  0  1  3  1  6  6  6  1
  1  6  7  1  7  6  5  7  0  7  1  5]
[[-1.7013587  -1.4184573  -1.7441583  ... -1.8740095  -3.2402468
  -1.3617852 ]
 [-2.4



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2310, training_loss=0.5591705712405118, metrics={'train_runtime': 979.9779, 'train_samples_per_second': 18.858, 'train_steps_per_second': 2.357, 'total_flos': 1138159578790080.0, 'train_loss': 0.5591705712405118, 'epoch': 10.0})

In [ ]:
!nvidia-smi

Wed Mar 30 20:39:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W /  70W |   4854MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------